In [ ]:
!pip install datasets
!pip install transformers

     |████████████████████████████████| 290 kB 8.3 MB/s 
     |████████████████████████████████| 243 kB 73.8 MB/s 
     |████████████████████████████████| 125 kB 66.0 MB/s 
     |████████████████████████████████| 1.1 MB 57.4 MB/s 
     |████████████████████████████████| 59 kB 9.1 MB/s 
     |████████████████████████████████| 192 kB 64.2 MB/s 
     |████████████████████████████████| 160 kB 73.2 MB/s 
     |████████████████████████████████| 271 kB 74.3 MB/s 
     |████████████████████████████████| 3.1 MB 7.2 MB/s 
     |████████████████████████████████| 895 kB 50.3 MB/s 
     |████████████████████████████████| 596 kB 92.7 MB/s 
     |████████████████████████████████| 3.3 MB 83.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import json
import random

from glob import glob

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import torch

from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer, EarlyStoppingCallback)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset

# model_ckpt = "bert-base-uncased"
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    ).to('cuda')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
!cp drive/MyDrive/libraries/* ./

cp: -r not specified; omitting directory 'drive/MyDrive/libraries/datasets_biblioteki'


In [ ]:
books_orig = pd.read_json("books.jsn")
books = pd.read_json("books.jsn")
for col in books.columns:
    books[col] = books[col].fillna("")
books["serial_name"] = books["serial_name"].apply(lambda x: x.lower())
books["material_name"] = books["serial_name"].apply(lambda x: x.lower())
books["rubric_name"] = books["serial_name"].apply(lambda x: x.lower())
books["main"] = (books["author_fullName"].str.lower() +
                 " " +
                #  ";" +
                 books["title"].apply(
                     lambda x: x.lower().split(":")[0].strip()
                 ))
books["id"] = books["id"].astype(int)
# group by author + book name
books = (books.groupby(["main"])
    .agg(
        bbk=("bbk", "first"),
        title=("title", "first"),
        serial_name=("serial_name", "max"),
        material_name=("material_name", "max"),
        annotation=("annotation", "max"),
        rubric_name=("rubric_name", "max"),
        author_fullName=("author_fullName", "first"),
        ids=(
            "id",
            lambda x: set(x),
        ),
    )
    .reset_index()
)

books["text"] = (books["main"] + " " + #";" +
                 books["rubric_name"].str.lower() + " " + #";" +
                #  books["serial_name"].str.lower() + "[SEP]" +
                #  books["material_name"].str.lower() + "[SEP]" +
                 books["annotation"].str.lower())
books.to_csv("text_csv.csv", index=None)
# books.head(2)

In [ ]:
vectors = []
decoded = []
less = 0
for text in books["text"].values:
    if type(text) != str:
        text = ""
    input_ids = tokenizer(text)["input_ids"][1:102]
    vectors.append(input_ids)
    decoded.append(tokenizer.decode(input_ids))
books["vectors"] = vectors
books["decoded"] = decoded

In [ ]:
books_dict = dict()
vectors_dict = dict()
for i, row in books.iterrows():
    ids = row["ids"]
    for book_id in ids:
        books_dict[book_id] = row["text"]
        vectors_dict[book_id] = row["vectors"]

In [ ]:
print(books["text"].str.len().max())
books["text"].values[0]

3041


' "божественное свидание и прочий флирт"  '

In [ ]:
# books["decoded"].values[6]

In [ ]:
# !iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/siglas.csv | head

In [ ]:
# !iconv -f cp1251 < drive/MyDrive/libraries/datasets_biblioteki/datasets_2/cat_1.csv | head

In [ ]:
# cat_2 = pd.read_csv("drive/MyDrive/libraries/"
#                     "datasets_biblioteki/datasets_2/cat_2.csv",
#                     encoding="cp1251",
#                     sep=";"
# )

In [ ]:
# cat_2[cat_2["recId"] == 1389555]

In [ ]:
# cat_2[cat_2["recId"] == 1814545]

In [ ]:
interactions = pd.read_excel("drive/MyDrive/libraries/dataset_knigi_1.xlsx")
interactions["book_id"] = interactions["source_url"].str.split("/").str[-2].astype(int)
interactions["book_str"] = interactions["book_id"].replace(books_dict)
print(interactions["event"].unique())
interactions.head()

['add' 'create_order']


,source_url,event,dt,user_id,book_id,book_str
0,https://www.mos.ru/knigi/book/1163786/,add,2021-06-01,1,1163786,бэйли гвен о чем думает ваша собака? книга со...
1,https://www.mos.ru/knigi/book/66612/,add,2021-06-01,1,66612,яковлев юрий яковлевич у человека должна быть ...
2,https://www.mos.ru/knigi/book/1034596/,add,2021-06-01,1,1034596,"баранова марта петровна тяпа, борька и ракета ..."
3,https://www.mos.ru/knigi/book/1152760/,add,2021-06-01,1,1152760,"дункан дэвид дуглас лумп - собака, которая съе..."
4,https://www.mos.ru/knigi/book/16168/,add,2021-06-01,1,16168,брюгген мария ван почему собака лает и еще 100...


In [ ]:
interact_vectors = []
for i, row in interactions.iterrows():
    book_id = row["book_id"]
    if book_id not in vectors_dict:
        vector = []
    else:
        vector = vectors_dict[row["book_id"]]
    interact_vectors.append(vector)
interactions["vector"] = interact_vectors

по этому единственному пользователю
насколько я понимаю из circulaton 
нас интересует только статус "На руках",
но книжек в циркулатоне даже для этого пользователя
больше

не понимаю, почему не всегда книжки из мос.ру в циркулатоне

In [ ]:
test_df = pd.read_csv("test.csv")
full = pd.read_csv("user_book_interaction_full.csv")
train_user_book = pd.read_csv("user_book_interaction_train.csv")
test_df

,user_id,book_id_1,book_id_2,book_id_3,book_id_4,book_id_5
0,34,1866509,1882247,1812326,2523335,1205629
1,23,80548,967568,1897994,2011853,734767
2,51,318612,16371,1201809,313152,349614
3,64,1824182,384273,1609359,836758,1824122
4,30,331155,6532,1791079,2002568,1818924
...,...,...,...,...,...,...
84,68,814603,828096,118466,1624999,2088574
85,28,139354,453682,9410,9481,13954
86,73,1800154,1961841,322503,841969,218331
87,88,374514,173502,346259,173584,458511


In [ ]:
print(interactions.shape, full.shape, train_user_book.shape, test_df.shape)

(4186, 7) (4186, 3) (3683, 3) (89, 6)


In [ ]:
from collections import Counter

test_indices = []
for i, row in test_df.iterrows():
    for value in row[1:]:
        test_index = interactions[
            (interactions["user_id"] == row["user_id"]) &
            (interactions["book_id"] == value)].index[0]
        test_indices.append(test_index)
print(len(test_indices))

445


In [ ]:
train = interactions.loc[interactions.index.difference(test_indices)]
test = interactions.loc[test_indices]

In [ ]:
train_dataset = []

In [ ]:
tokenizer("[CLS]")

{'input_ids': [101, 101, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [ ]:
def get_history_vector(user_dt_books, sample_size=4, sep_token=102):
    joint_vector = []
    if user_dt_books.shape[0] < sample_size:
        sample_size = user_dt_books.shape[0]
    history_vectors = random.sample(
        list(user_dt_books["vector"].values), sample_size)
    for h in history_vectors:
        # if joint_vector and joint_vector[-1] != sep_token:
        #     joint_vector.append(sep_token)
        joint_vector += h
    if not joint_vector:
        print(user_dt_books["vector"])
        print(history_vectors)
        print(joint_vector)
    if joint_vector[-1] != sep_token:
        joint_vector.append(sep_token)
    joint_vector = [101] + joint_vector
    return joint_vector

j = 0
for i, row in train.iterrows():
    print(f"\r{j}", end="")
    j += 1
    user_id = row["user_id"]
    dt = row["dt"]
    user_dt_books = train[
        (train["user_id"] == user_id) &
        (train["dt"] == dt) &
        (train["event"] == "create_order")]
    user_dt_books = user_dt_books[
        user_dt_books["vector"].apply(lambda x: len(x) > 0)]
    if user_dt_books.shape[0] == 0:
        continue
    next_dates = train[
        (train["user_id"] == user_id) &
        (train["dt"] > dt) &
        (train["event"] == "create_order")]["dt"].astype(str).unique()
    if next_dates.shape[0] == 0:
        continue
    next_date = next_dates[0]
    next_book_slice = train[
        (train["user_id"] == user_id) &
        (train["dt"] == next_date)]
    next_book_indices = set(next_book_slice["book_id"].values)
    other_books = books[books["ids"].apply(
        lambda x : len(x.intersection(next_book_indices)) == 0)]
    for vector in next_book_slice["vector"].values:
        for i in range(1):
            joint_vector = get_history_vector(user_dt_books)
            joint_vector += list(random.choice(
                other_books["vectors"].values))
            train_dataset.append([joint_vector, 0])
        joint_vector = get_history_vector(user_dt_books)
        joint_vector += list(vector)
        train_dataset.append([joint_vector, 1])

3740

In [ ]:
random.shuffle(train_dataset)

In [ ]:
print(tokenizer.decode(train_dataset[1][0]))

[CLS] говоров александр алексеевич последние каролинги золотои треугольник раннее средневековье, франция второи половины ix века. идет уаилдер лора инглз долгая зима ; городок в прерии мир девочек [SEP] ивич александр приключения изобретении любое изобретение начинается с опыта, жизненного и научного : первооткрыватель д [SEP] искандер фазиль абдулович тринадцатыи подвиг геракла классика в школе в книгу включены произведения ф. а. искандера, которые и


In [ ]:
max_text_len = max(len(t[0]) for t in train_dataset)
print(max_text_len)

507


In [ ]:
for t in train_dataset:
    if len(t[0]) < max_text_len:
        t[0] += [0] * (max_text_len - len(t[0]))
print(max(len(t[0]) for t in train_dataset))
print(min(len(t[0]) for t in train_dataset))

507
507


In [ ]:
train_dict = {"input_ids": [t[0] for t in train_dataset],
              "label": [t[1] for t in train_dataset]}
dataset = Dataset.from_dict(train_dict)
dataset = dataset.train_test_split(0.05)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(labels[:2])
    print(predictions[:2])
    predictions = np.argmax(predictions, axis=1)
    f1_results = f1_score(labels,
                          predictions, average="micro")
    output = {'accuracy': accuracy_score(labels, predictions),
              'f1': f1_results}
    print(output)
    return output

batch_size = 12
args = TrainingArguments(
    output_dir="library_model2",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=100,
    weight_decay=0.01,
    warmup_steps=2000,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)


trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=20)]
    )

# trainer.evaluate()
trainer.train()

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 5432
  Num Epochs = 100
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 45300


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.692300,0.707552,0.482517,0.482517
1000,0.688300,0.696234,0.506993,0.506993
1500,0.680000,0.689009,0.566434,0.566434
2000,0.663700,0.728598,0.597902,0.597902
2500,0.598200,0.696758,0.646853,0.646853
3000,0.504500,0.683036,0.699301,0.699301
3500,0.399000,0.680313,0.748252,0.748252
4000,0.341000,0.537788,0.804196,0.804196
4500,0.281400,0.553519,0.800699,0.800699
5000,0.251800,0.716856,0.811189,0.811189


***** Running Evaluation *****
  Num examples = 286
  Batch size = 12


[0 0]
[[-0.04060377 -0.0856173 ]
 [-0.2563676   0.0921789 ]]
{'accuracy': 0.4825174825174825, 'f1': 0.4825174825174825}


Saving model checkpoint to library_model2/checkpoint-500
Configuration saved in library_model2/checkpoint-500/config.json
Model weights saved in library_model2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in library_model2/checkpoint-500/tokenizer_config.json
Special tokens file saved in library_model2/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 286
  Batch size = 12


...

[0 0]
[[ 3.628165  -3.5667794]
 [ 5.8265176 -6.3882604]]
{'accuracy': 0.8041958041958042, 'f1': 0.8041958041958042}


Saving model checkpoint to library_model2/checkpoint-23000
Configuration saved in library_model2/checkpoint-23000/config.json
Model weights saved in library_model2/checkpoint-23000/pytorch_model.bin
tokenizer config file saved in library_model2/checkpoint-23000/tokenizer_config.json
Special tokens file saved in library_model2/checkpoint-23000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 286
  Batch size = 12


[0 0]
[[-4.6981606  4.743397 ]
 [ 6.1216063 -6.808475 ]]
{'accuracy': 0.8251748251748252, 'f1': 0.8251748251748252}


Saving model checkpoint to library_model2/checkpoint-23500
Configuration saved in library_model2/checkpoint-23500/config.json
Model weights saved in library_model2/checkpoint-23500/pytorch_model.bin
tokenizer config file saved in library_model2/checkpoint-23500/tokenizer_config.json
Special tokens file saved in library_model2/checkpoint-23500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from library_model2/checkpoint-13500 (score: 0.8846153846153846).


TrainOutput(global_step=23500, training_loss=0.16365202006887883, metrics={'train_runtime': 15979.1124, 'train_samples_per_second': 33.994, 'train_steps_per_second': 2.835, 'total_flos': 7.341958394298216e+16, 'train_loss': 0.16365202006887883, 'epoch': 51.88})

In [ ]:
!zip library_model.zip library_model2/checkpoint-13500/*
!cp library_model.zip drive/MyDrive/libraries

  adding: library_model2/checkpoint-13500/config.json (deflated 49%)
  adding: library_model2/checkpoint-13500/optimizer.pt (deflated 27%)
  adding: library_model2/checkpoint-13500/pytorch_model.bin (deflated 7%)
  adding: library_model2/checkpoint-13500/rng_state.pth (deflated 27%)
  adding: library_model2/checkpoint-13500/scheduler.pt (deflated 49%)
  adding: library_model2/checkpoint-13500/special_tokens_map.json (deflated 40%)
  adding: library_model2/checkpoint-13500/tokenizer_config.json (deflated 39%)
  adding: library_model2/checkpoint-13500/tokenizer.json (deflated 59%)
  adding: library_model2/checkpoint-13500/trainer_state.json (deflated 83%)
  adding: library_model2/checkpoint-13500/training_args.bin (deflated 48%)
  adding: library_model2/checkpoint-13500/vocab.txt (deflated 53%)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "library_model2/checkpoint-13500",
    ).to('cuda')

loading configuration file library_model2/checkpoint-13500/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file library_model2/checkpoint-13500/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights o

In [ ]:
token_type_ids = torch.from_numpy(
    np.zeros(
        shape=max_text_len)
    ).to("cuda").to(torch.int32)
attention_mask = torch.from_numpy(
    np.ones(shape=(1,max_text_len))).to("cuda").to(torch.int32)

In [ ]:
user_preds = []
token_type_ids
for user in train["user_id"].unique():
    user_slice = train[train["user_id"] == user]
    prev_books = set(user_slice["book_id"].values)
    user_vectors = list(user_slice["vector"].values[-4:])
    joint_vector = []
    for u in user_vectors:
        joint_vector += u
    for i, row in books.iterrows():
        # prev = False
        # for b_id in row["ids"]:
        #     if b_id in prev_books:
        #         prev = True
        # if prev:
        #     continue
        book_vector = joint_vector + row["vectors"]
        book_vector += [0] * (max_text_len - len(book_vector))
        book_array = np.array([book_vector])
        book_torch_v = torch.from_numpy(book_array).to("cuda")

        input_v = {'input_ids': book_torch_v,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}
        logits = model(**input_v)[0]
        preds = logits.cpu().detach().numpy()[0]
        pred_class = np.argmax(preds)
        user_preds.append([user, i, pred_class, preds[0], preds[1]])
        print(user_preds[-1])

Streaming output truncated to the last 5000 lines.
[102, 3322, 0, 1.6454396, -1.4632822]
[102, 3323, 1, -1.4680257, 1.5221982]
[102, 3324, 1, -0.23860562, 0.25292674]
[102, 3325, 1, -1.4055681, 1.3129168]
[102, 3326, 1, -2.3433666, 2.120585]
[102, 3327, 1, -2.1118655, 1.965755]
[102, 3328, 0, 3.6048536, -3.627186]
[102, 3329, 0, 4.3776245, -4.5398855]
[102, 3330, 0, 1.4421555, -1.0882185]
[102, 3331, 0, 0.9995725, -0.7146207]
[102, 3332, 0, 4.8406234, -4.783324]
[102, 3333, 0, 4.2044415, -4.1245246]
[102, 3334, 1, -0.65269864, 0.8712847]
[102, 3335, 0, 2.969554, -2.7596257]
[102, 3336, 0, 2.285515, -1.8868858]
[102, 3337, 0, 4.475622, -4.5293975]
[102, 3338, 0, 0.60908914, -0.5724169]
[102, 3339, 0, 5.1327877, -5.424838]
[102, 3340, 0, 3.5210378, -3.2210836]
[102, 3341, 0, 4.80403, -4.840561]
[102, 3342, 0, 3.699484, -3.5047772]
[102, 3343, 0, 3.3863685, -3.5416908]
[102, 3344, 0, 1.6449065, -1.1843843]
[102, 3345, 1, -2.8666487, 2.6166735]
[102, 3346, 0, 1.6046147, -1.2221977]
[102, 3

In [ ]:
user_df = pd.DataFrame(
    user_preds,
    columns=["user", "book", "pred_class", "logits_0", "logits_1"])

In [ ]:
user_df.head()

,user,book,pred_class,logits_0,logits_1
0,1,0,0,5.078056,-5.396259
1,1,1,0,5.143651,-5.635192
2,1,2,0,3.785104,-3.405183
3,1,3,0,3.721545,-3.485675
4,1,4,0,5.066837,-5.233332


In [ ]:
user_df[user_df["pred_class"] == 1].head()

,user,book,pred_class,logits_0,logits_1
8,1,8,1,-3.628496,3.340718
24,1,25,1,-1.250339,1.582186
28,1,29,1,-2.210876,2.666790
35,1,36,1,-0.285577,0.487359
40,1,41,1,-0.849712,1.512381


In [ ]:
user_df[user_df["book"] == 8].head()

,user,book,pred_class,logits_0,logits_1
8,1,8,1,-3.628496,3.340718
4046,2,8,1,-3.413481,3.199259
8147,3,8,0,5.076566,-5.205732
12299,4,8,1,-2.996668,2.436995
16452,5,8,1,-3.906426,3.806929


In [ ]:
user_df.to_csv("user_df2.csv", index=None)

In [ ]:
!cp user_df2.csv drive/MyDrive/libraries

In [ ]:
def mean_average_precision_k(predictions: list, targets: list):
    """ Calculate mAP@k.
    :param predictions: list of predictions (id of book) for each user
    :param targets: list of target (id of book) for each user
    :return: current metric mAP@k type float
    """
    if not targets or not predictions:
        raise ValueError("Empty data.")

    map_k = []

    for prediction, target in zip(predictions, targets):
        max_tp = len(target)
        ap = 0
        tp = 0
        for pred_value, index in zip(prediction, range(len(prediction))):
            if tp == max_tp:
                break
            if pred_value in target:
                tp += 1
                ap += tp / (index + 1)
        ap /= max_tp
        map_k.append(ap)

    return np.mean(map_k)

In [ ]:
doc_freq = dict()
for book in user_df["book"].unique():
    doc_freq[book] = user_df[(user_df["book"] == book) &
                             (user_df["pred_class"] == 1)].shape[0]
user_df["doc_freq"] = user_df["book"].replace(doc_freq) + 1
user_df["idfed"] = user_df["logits_1"] / user_df["doc_freq"]

In [ ]:
predictions = []
targets = []
for user in test_df["user_id"]:
    # print(user)
    done = set(train[train["user_id"] == user]["book_id"])
    sliced = user_df[user_df["user"] == user]
    book_ids = list(sliced.sort_values("logits_1")["book"].values)
    real_book_ids = []
    for book in book_ids[::-1]:
        real_book = list(books["ids"].values[book])
        # if real_book[0] in done:
        #     continue
        done.update(set(real_book))
        # print(books.values[book][0])
        real_book_ids.append(real_book[0])
        if len(real_book_ids) >= 5:
            break
    target_slice = test_df[test_df["user_id"] == user]
    if target_slice.shape[0] != 0:
        user_targets = list(target_slice.values[0][1:])
        predictions.append(real_book_ids)
        targets.append(user_targets)
    else:
        raise Exception
    # print("-"*80)
    # print(user, book_ids)

mean_average_precision_k(predictions, targets)

0.0

In [ ]:
print(targets[0])
print(test_df.values[0])

[1866509, 1882247, 1812326, 2523335, 1205629]
[     34 1866509 1882247 1812326 2523335 1205629]


In [ ]:
for p in predictions[0]:
    print(books_dict[p][:80])
print("-" * 80)
for p in targets[0]:
    print(books_dict[p][:80])

print(books[books["text"].apply(lambda x: "ледерман виктория валерьевна светлик" in x)])

плещеев алексей николаевич и сердца жар и чистые стремленья...  
толстой лев николаевич детям точка, точка, запятая... 
юхансон георг история самолетов. рассказывает  мулле мек  
петрановская людмила владимировна что делать,если...  
говоров александр алексеевич последние каролинги золотой треугольник раннее сред
--------------------------------------------------------------------------------
линдгрен астрид калле блумквист и расмус любимое чтение это заключительная часть
линдгрен астрид калле блумквист и расмус любимое чтение это заключительная часть
ледерман виктория валерьевна светлик тучкин и пузырь желаний  школьная жизнь све
ледерман виктория валерьевна светлик тучкин и украденные каникулы  вы только пре
 белогрудка правдивые истории в сборник вошли рассказы виктора астафьева, алекса
                                                   main  ...                                            decoded
2109  ледерман виктория валерьевна светлик тучкин и ...  ...  ледерман виктория валерье

In [ ]:
user_df[(user_df["user"] == 34) &
        (user_df["book"] >= 25) &
        (user_df["book"] <= 25)]

,user,book,pred_class,logits_0,logits_1,doc_freq,idfed
131613,34,25,1,-2.807784,2.833537,51,0.05556
